# Python for Data Analytics - Exercises 5 solutions

# Linear Classification

## Objectives

* To gain some practical experience with linear classification.
* To reduce Bayesian classifiers to linear classifiers by constraining the covariances of the classes to be equal.
* To compare the performance of a generative model and a discriminative model on a challenging classification task.

## 1. Background

In these exercises we will be revisiting the Bayesian classification code that we used in the previous exercises but we will constrain the system to have equal covariance matrices effectively turning it into a [linear classifier](https://en.wikipedia.org/wiki/Linear_classifier). We will then compare this 'generative’ approach with the 'discriminative’ approach: learning the linear classifier parameters directly by using the [perceptron](https://en.wikipedia.org/wiki/Perceptron) learning algorithm.

## 2. Introduction

In this notebook we will be using another data set from the UCI machine-learning repository: abalone data. An abalone is a type of sea snail. The age of a specimen can be determined by cutting the shell through the cone and counting rings through a microscope (rather like trees), but this is a time consuming and expensive procedure. The task here is to predict the number of rings given simple external measurements of the weight and dimension of the animal. For the data set we are using the real values are known (i.e. the rings were counted after the snails were measured). Results vary from 1 to 29 rings, so this would usually be treated as a 29-class classification problem. To simplify things a little I have regrouped the data into just two classes of roughly equal size: young (less than 10 rings) and old (10 or more rings). I have also only taken the female samples. There are 7 measurements (which are all quite highly correlated) that are to be used to predict the class label. (The precise meaning of the measurements can be found here http://archive.ics.uci.edu/ml/machine-learning-databases/abalone/abalone.names)

Compared to the wine classification task this task is quite challenging. It will be impossible to get 100% correct because the classes are not [linearly separable](https://en.wikipedia.org/wiki/Linear_separability). Further, most of the specimens have either 8, 9, 10 or 11 rings and so lie close to the young/old borderline. However, you should be able to get percentage correct scores that are considerably higher than the 50% that would be expected by guessing alone.

## 3. Obtaining the data

The data can be read directly into a numpy array using the code in the cell below. The data will form a matrix with 1,306 rows and 8 columns. Each row is a separate sample (i.e. a different snail). The first column stores a class label (1 or 2). Columns 2 to 8 are the results of the 7 length and weight measurements. Note that, like in the last notebook, the features are stored as columns and the samples as rows. This makes things easier when using Python but the opposite convention is used.

In [ ]:
import numpy as np
X = np.loadtxt(open("files/abalone.txt","r"))
X.shape

## 4. Generative modeling: Bayesian classification with equi-covariant multivariate normal distributions.

Compared to last notebook, there are a lot more samples (1,306 compared with 178) so we will not worry about leave-one-out testing, instead, we will simply cut the data into equal sized testing and training sets like we did for the first half of the last notebook.

By adapting the code you wrote last time, evaluate the performance of a Bayesian classifier using multivariate normal distributions with full covariance matrices. When considering changes to the code note that the main difference is that in this notebook there are only two classes rather than three. (If you wish you may trying wrapping the code into a function and seeing if you can design it so that it works for any number of classes.)

How well does your classifier perform? (The score will probably be in the 60-70% range for this task, so don’t worry if performance seems a lot poorer than last notebook).

In [ ]:
# SOLUTION
from scipy.stats import multivariate_normal
import matplotlib.pyplot as plt
%matplotlib inline

abalone1 = X[X[:,0]==1,:]
abalone2 = X[X[:,0]==2,:]
abalone1_test = abalone1[0::2,:]
abalone1_train = abalone1[1::2, :]
abalone2_test = abalone2[0::2,:]
abalone2_train = abalone2[1::2, :]
abalone_test = np.vstack((abalone1_test, abalone2_test))
abalone_test.shape

mean1 = np.mean(abalone1_train[:,1:], axis=0)
mean2 = np.mean(abalone2_train[:,1:], axis=0)
cov1 = np.cov(abalone1_train[:,1:], rowvar=0)
cov2 = np.cov(abalone2_train[:,1:], rowvar=0)

dist1 = multivariate_normal(mean=mean1, cov=cov1)
dist2 = multivariate_normal(mean=mean2, cov=cov2)

p1=dist1.pdf(abalone_test[:,1:])
p2=dist2.pdf(abalone_test[:,1:])

p = np.vstack((p1,p2))

index = np.argmax(p, axis=0)+1

plt.plot(index, 'k.', ms=10)

correct = abalone_test[:,0]==index
percent_correct = np.sum(correct)*100.0/index.shape
print(percent_correct)

**Using equal covariance matrices.**

If you correctly followed the same procedures as last notebook you will have estimated a separate covariance matrix for each class. These matrices will not be equal and so the system will not have been a linear classifier. In order to reduce it down to a linear system we need to ensure that there is only one covariance matrix. There are different ways that you might imagine doing this:

First, you might imagine simply estimating a single covariance matrix from the complete training set before dividing it into classes. This produces a single matrix but it is not the correct thing to do. We want the matrix to represent the spread *within* the classes, if you simply train a model using the full training data set it will also be capturing the spread *between* the classes.

Second, you could imagine *averaging* the two class-dependent covariance matrices. This is closer to the correct thing but it doesn’t take care of the fact that the classes might have had unequal numbers of examples. The best approach is to first move the centres of the two classes onto the same point and then treat them as a single class. To move the class centres onto the same point you can simply subtract the class mean vector from each data sample.

E.g., for each sample in class one you need to compute,
    
    abalone1_train(i,:)– mean1
    
and for each sample in class two you would compute,
    
    abalone2_train(i,:) – mean2

You can do this without a loop. In fact you can just type
  
    abalone2_train - mean2
  
When numpy sees a vector being subtracted from a matrix like this, it will automatically copy the vector so that it is subtracted from every row of the matrix (this is called 'broadcasting').

Go ahead and modify your code so that a single covariance matrix is computed called, for example, cov_global.

In [ ]:
# SOLUTION

def centre_data(data):
    nsamples = data.shape[0]
    data_mean = np.mean(data, axis=0)
    data_centred = data - data_mean
    return data_centred

abalone1_centred = centre_data(abalone1_train)
abalone2_centred = centre_data(abalone2_train)

abalone_centred = np.vstack((abalone1_centred, abalone2_centred))

cov_global = np.cov(abalone_centred[:,1:], rowvar=0)

You can now redo the classification using this single matrix in both mvnpdf evaluations
    
    p1 = mvnpdf(abalone1_test, mean1, cov_global);
    p2 = mvnpdf(abalone2_test, mean2, cov_global);
    etc

What is the new result? How does it compare with the result when using the more flexible non-linear classifer (i.e. when using two different covariance matrices)?

In [ ]:
dist1 = multivariate_normal(mean=mean1, cov=cov_global)
dist2 = multivariate_normal(mean=mean2, cov=cov_global)

p1=dist1.pdf(abalone_test[:,1:])
p2=dist2.pdf(abalone_test[:,1:])

p = np.vstack((p1,p2))

index = np.argmax(p, axis=0)+1

plt.plot(index, 'k.', ms=10)

correct = abalone_test[:,0]==index
percent_correct = np.sum(correct)*100.0/index.shape
print(percent_correct)

If we wished we could represent this linear Gaussian classifier in the standard linear classifier form, i.e. $g(x) = w’x+w_0$ and output $\omega_1$ if $g(x)>0$ else $\omega_2$. The parameters $w’$ and $w_0$ would simply be a function of the covariance matrix and the class means that we have learnt from the training data. Actually, although this would involve an extra bit of calculation, once we had computed $w’$ and $w_0$, all the test points could be classified very quickly by evaluating $g(x)$ directly (e.g., for an $N$-dimension problem this amounts to just $N+1$ multiplications, $N$ additions and a comparison against 0).

## 5. Disciminative modeling: the Perceptron learning algorithm ##

In the 2nd half of this notebook we turn to look at a discriminative approach to classification in which the $w$ parameters needed for the evaluation of $g(x)$ are learnt directly. We will use the [perceptron learning algorithm](https://en.wikipedia.org/wiki/Perceptron); the pseudocode for this algorithm is roughly as follows:

1. Initialize weights ${\bf w}$ and thresholds
2. For each example $i$ in the training set $X$, perform the following over the input ${\bf x}_i$ and the class labels ${\bf y}_i$:
    1. Calculate the actual output $o_i(t) = f({\bf w}(t) * {\bf x}_i)$
    2. Update the weights $w_i(t + 1) = w_i(t) - (y_i - o_i(t))x_{i,j}$ for all features $0 \leq j \leq \mbox{n}$
3. Repeat step (2) until the iteration error is smaller than a given error threshold (i.e. number of misclassifications is 'low enough'), or a pre-determined number of iterations were executed

The perceptron learning algorithm is quite easy to implement, but in order to save time I’ve provided an implementation for you below. The function has several inputs: training data, training_labels, an initial guess at the weights and a learning rate. **Note, the class labels have to be given the values +1 and -1 for the two classes**. It will return a tuple containing:
<ol>
    <li>learnt w parameters</li>
    <li>the number of iterations performed</li>
    <li>the number of misclassfied samples</li>
</ol>

Take some time to examine the code. Don’t worry if it is not immediately clear how each line is working, but just satisfy yourself that you understand what each line is meant to be doing.

In [ ]:
def perce(X, y, w_init, rho):
    """ perce
    A simple but inefficient implementation of the perceptron learning alogorithm

    X - the data matrix. Each row represents a separate sample
    y - a vector of integer class labels corresponding to the rows of X - labels must be +1 or -1
    w_init - the initial weight vector 
    rho - a scalar learning rate
    """

    (N, nfeatures) = X.shape
    
    # Augment the feature vectors by adding a 1 to each one. (see lecture notes)
    X = np.hstack((X, np.ones((N,1))))
    nfeatures += 1
    
    max_iter = 100  # maximum number of iterations
    w = w_init      # initialise weights
    iter = 0      
    mis_class = N   # start by assuming all samples are misclassified

    while mis_class > 0 and iter < max_iter:
        iter += 1
        mis_class = 0
        gradient = np.zeros((1,nfeatures)) # initaliase the gradients to 0
        
        # loop over every training sample. 
        for i in range(N):
            # each misclassified point will cause the gradient to change
            if np.inner(X[i,:], w)*y[i] >= 0:
                mis_class += 1
                gradient += y[i] * X[i,:]
        # update the weight vector ready for the next iteration        
        w -= rho*gradient
        
    return w, iter, mis_class

The implementation above is quite straightforward. Unfortunately, the inner loop over the samples is very slow. This is largely because Python is an interpretted language. We can speed it up by processing all the points at the same time by using matrix operations than are processed by numpy. The improved implementation is shown below. Note how the inner loop has now 'vanished'. This version runs 100 times faster! Note that the max_iter has been set to 10000 rather than 100 so that up to 100 times as many iteraions may be performed.

In [ ]:
def perce_fast(X, y, w_init, rho):
    """ perce_fast
    A more efficient implementation of the perceptron alogorithm
    For the notebook data this version will work x100 faster!

    X - the data matrix. Each row represents a separate sample
    y - a vector of integer class labels corresponding to the rows of X - labels must be +1 or -1
    w_init - the initial weight vector 
    rho - a scalar learning rate
    """
    (N, nfeatures) = X.shape
    X = np.hstack((X, np.ones((N,1))))
    nfeatures += 1
    max_iter = 10000
    w = w_init
    iter = 0
    mis_class = N
    yy = np.tile(y, (1, nfeatures))
    while mis_class > 0 and iter < max_iter:
        iter += 1
        mis_class = 0
        gradient = np.zeros((1,nfeatures))
        mc = ((np.dot(X, w.transpose()) * y) >= 0)[:,0]
        mis_class = np.sum(mc)
        w -= rho * (np.sum(yy[mc, :] * X[mc, :], axis=0))
    return w, iter, mis_class

Now use perceptron function with the same training data that you used previously in order to learn the weights. You will need to read the documentation at the head of the function so that you understand what inputs are required. In particular, note that the class labels have to be in the form of a vector storing -1’s and +1’s. So you will have to manipulate your training data a little because it currently has the classes labeled represented as 1 and 2.

Experiment with different learning rates and different numbers of iterations. The function returns the number of errors that are made on the training set. You want this number to be as low as possible.

In [ ]:
# SOLUTION
abalone_train = np.vstack((abalone1_train, abalone2_train))
abalone_train_data = abalone_train[:, 1:]
abalone_train_labels = np.array([abalone_train[:, 0]]).transpose() * 2 - 3

#rho = 1
#w_init = np.zeros((1, abalone_train_data.shape[1]+1))
#w, iter, mis_class = perce(abalone_train_data, abalone_train_labels, w_init, rho)
#print(w)
#print(mis_class)

rho = 0.1
w_init = np.zeros((1, abalone_train_data.shape[1]+1))
w, iter, mis_class = perce_fast(abalone_train_data, abalone_train_labels, w_init, rho)
print(w)
print(mis_class)

**Evaluating the classifier:** You now need to evaluate the $w$ vector that the learning algorithm has produced. To do this you will need to evaluate $w’x+w_0$ for each element in the test set and generate a label by comparing the result against 0. Then compute the percentage of labels that match the correct test set labels. It is probably best to write a little function to do this.

In [ ]:
# SOLUTION

# Make test data and test labels
abalone_test = np.vstack((abalone1_test, abalone2_test))
abalone_test_data = abalone_test[:, 1:]
abalone_test_labels = np.array([abalone_test[:, 0]]).transpose() * 2 - 3

# Perform classification - copied from perce_fast
X = abalone_test_data
(N, nfeatures) = X.shape
X = np.hstack((X, np.ones((N,1))))
mc = ((np.dot(X, w.transpose()) * abalone_test_labels) >= 0)[:,0]
mis_class = np.sum(mc)

# Convert number of miss classifications into percentage correct
print (100.0 - mis_class*100.0/N)

How well does the new classifier perform? How does performance compare with the linear classifier that you built using Gaussians in the first half of the notebook?

## 6. Notes

You should find that although the new classifier is again linear, its performance is a little better. You might be confused about this because it is said that Bayesian classifiers are optimal – their decision boundaries are positioned so as to minimize the number of classification errors. However, this is only true in the theoretical sense that they are optimal if we use the correct distribution for $p(x|w)$. In practice we don’t know this distribution so we estimate it from the training data. This is typically done by assuming $p(x|w)$ has some general shape (e.g. Gaussian) and then estimating specific parameters, e.g. means and covariances, that best fits the training data. This would be fine is our initial assumption was correct, but the fact is that the data is very unlikely to fit some nice well-behaved distribution like a Gaussian. For example, it might be approximately Gaussian but slightly skewed, or with slightly elongated tails etc etc. So even with the best possible parameter tuning there will still exist a discrepancy between the $p(x|w)$ that we’ve estimated and the true distribution. If we aren’t using the correct $p(x|w)$ then all bets are off and we can’t guarantee that the model we have chosen will be the one that minimizes errors.

The discriminative approach on the other hand is doing something that is closer to directly minimizing errors. It is doing this without regard to the distribution of $p(x|w)$. This more pragmatic approach often works better particularly in cases where $p(x|w)$ has a complicated distribution that is hard to parameterize. It is also perhaps closer to what we do as humans, e.g. we are often not aware of small differences within classes (e.g. in speech, all ‘b’ phonemes sound roughly similar) but we are very sensitive to small changes at the borders between classes (‘b’ and ‘p’ are perceptually quite different, but if you look at features extracted from their sound signals they can be extremely similar).

# Programming Challenge

## 7. Programming exercise

Two versions of the Perceptron learning algorithm are discussed. The first works in batch mode like the code you have used here, i.e. all points are first classified and then the weights are updated. The second, works in an online mode and processes the samples one at a time, i.e. the w parameters are adjusted directly after processing each sample.

Copy the function perce() into the cell below and then rewrite it so that it works in the online manner. (Note, unfortunately, the fast version perce_fast cannot be made to run in an online manner because it gets its speed precisely because it processes all samples in one step.) Does it produces better results for the Abalone task? 